In [108]:
import pandas as pd
import numpy as np

In [109]:
data = pd.read_csv("/Users/lucastucker/REU-2023/archive/mnist_train.csv")
data = np.array(data)
np.random.shuffle(data)

In [110]:
data.shape

(60000, 785)

In [111]:
m = 2000
N = 784
sample = data[:m]
X = sample[:, 1:]
X.shape

(2000, 784)

In [118]:
k = 100 # k nearest neighbors
k_nearest = np.ones((m, k)) * np.inf
p = 100 # projection size

In [113]:
diff_matr = X - X[0]
norms = np.linalg.norm(diff_matr, axis = 1)
print(norms)

[   0.         2337.87702842 1851.47400738 ... 1913.98484842 2320.39867264
 2159.18341046]


In [119]:
for id, row in enumerate(X):
    dif = X - row
    norm_indices = np.argsort(np.linalg.norm(dif, axis = 1))
    k_nearest[id] = norm_indices[1:k + 1]

In [120]:
A = np.random.normal(size = (p, N)) # random p projection
X_tilde = A.dot(X.T)
X_tilde = X_tilde.T
X_tilde.shape

(2000, 100)

In [122]:
k_nearest_rand = np.ones((m, k)) * np.inf
for id, row in enumerate(X_tilde):
    dif = X_tilde - row
    norm_indices = np.argsort(np.linalg.norm(dif, axis = 1))
    k_nearest_rand[id] = norm_indices[1:k+1]

In [123]:
shared_elems_list = []
for row, row_tilde in zip(k_nearest, k_nearest_rand):
    set_1 = set(row)
    set_2 = set(row_tilde)
    shared_elem_count = len(set_1.intersection(set_2))
    shared_elems_list.append(shared_elem_count)
shared_elems = np.array(shared_elems_list)
print(shared_elems) # ith element is ith row intersection count

[32 34 37 ... 32 21 26]


In [124]:
avg_shared = (1/m) * np.sum(shared_elems)
print(avg_shared)

34.593
